Zuveir Jameer

8 April 2025

Batch 1 which contains 4 search engines.

The code below are to run the 75 queries against an Elasticsearch instance which contains TREC 2021 Clinical Trials data.

The Elasticsearch instance contains ~26,000 indexed documents.

The default Elasticsearch indexing is BM25.

In the code below you will run searches using the following:
1. Semantic search on tokenised 'detailed_description' (detailed_descrption_vector) with ClinicalBERT
2. B25 as initial search. Then second search using semantic search to re-rank results
3. BM25 only
4. BM25W where weight as been added to the 'detailed_descrption'

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch, helpers
from credentials import USERNAME, PASSWORD
import json

In [2]:
CERT_FINGERPRINT = "24a71503f93ed6e422bc8782254a826382fadd83ca09234fa7c0297173132530"

In [3]:
# Connect using API KEY
es = Elasticsearch("https://localhost:9200",api_key="eHdSWmo1VUJOYkdYV0x0X3QyTnY6cU9kQVJ1Y0VRdmFWSTc5bzB6THoxUQ==",ssl_assert_fingerprint=CERT_FINGERPRINT,)

In [4]:
# Connect using credentials and ssl fingerprint
es = Elasticsearch("https://localhost:9200",basic_auth=(USERNAME, PASSWORD),ssl_assert_fingerprint=CERT_FINGERPRINT)

Check whether connected to ES

In [5]:
client_info = es.info()
print('Connected to elasticsearch')
pprint(client_info.body)

Connected to elasticsearch
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'daxPe0rzR9Gx5iOkHZsZTQ',
 'name': '200fd7a2e004',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-02-28T10:07:26.089129809Z',
             'build_flavor': 'default',
             'build_hash': 'a091390de485bd4b127884f7e565c0cad59b10d2',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.12.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.17.3'}}


### Semantic search only using ClinicalBERT 

(which was used to create token embeddings in the detailed_descrption_vector dense vector field)

In [ ]:
import json
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch
import time
import pandas as pd
from datetime import datetime
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModel.from_pretrained("medicalai/ClinicalBERT")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def mean_pooling(model_output, attention_mask):
    """
    Perform mean pooling on token embeddings while taking the attention mask into account.
    """
    token_embeddings = model_output.last_hidden_state  # (batch_size, seq_len, hidden_size)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
    return sum_embeddings / sum_mask

def get_embedding(text, max_length=512):
    """
    Compute the embedding for the given text using ClinicalBERT.
    Returns the embedding as a list of floats.
    """
    encoded_input = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        model_output = model(**encoded_input)
    # embedding_temp = mean_pooling(model_output, encoded_input['attention_mask'])
    embedding = mean_pooling(model_output, encoded_input['attention_mask'])
     # Normalize the embedding to unit length
    # embedding = F.normalize(embedding_temp, p=2, dim=1) #Added 22 March
    return embedding[0].cpu().numpy().tolist()


# Evaluation Metric Functions
def precision_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / k if k else 0.0

def recall_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    total_relevant = sum(1 for doc, rel in relevant_docs.items() if rel > 0)
    if total_relevant == 0:
        return 0.0
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / total_relevant

def f1_at_k(re_ranked_docs, relevant_docs, k):
    p = precision_at_k(re_ranked_docs, relevant_docs, k)
    r = recall_at_k(re_ranked_docs, relevant_docs, k)
    if (p + r) == 0:
        return 0.0
    return 2 * (p * r) / (p + r)

def average_precision(re_ranked_docs, relevant_docs):
    if not relevant_docs:
        return 0.0
    
    relevant_indices = [
        i for i, doc in enumerate(re_ranked_docs)
        if doc in relevant_docs and relevant_docs[doc] > 0
    ]
    if not relevant_indices:
        return 0.0
    
    ap_sum = 0.0
    for idx, rank_pos in enumerate(relevant_indices, start=1):
        prec = precision_at_k(re_ranked_docs, relevant_docs, rank_pos+1)
        ap_sum += prec
    return ap_sum / len(relevant_indices)

def ndcg_at_k(re_ranked_docs, relevant_docs, k):
    def dcg(scores):
        return sum(( (2**s - 1) / np.log2(idx + 2)) for idx, s in enumerate(scores))
    
    top_k_docs = re_ranked_docs[:k]
    actual_scores = [relevant_docs[doc] if doc in relevant_docs else 0 for doc in top_k_docs]
    actual_dcg = dcg(actual_scores)
    
    sorted_rels = sorted(relevant_docs.values(), reverse=True)[:k]
    ideal_dcg = dcg(sorted_rels)
    
    if ideal_dcg == 0:
        return 0.0
    return actual_dcg / ideal_dcg

def mrr_at_k(re_ranked_docs, relevant_docs, k):
    for i, doc_id in enumerate(re_ranked_docs[:k], start=1):
        if doc_id in relevant_docs and relevant_docs[doc_id] > 0:
            return 1.0 / i
    return 0.0

# Data Loading query and qrels from JSONL files
def load_jsonl(filepath):
    data = {}
    with open(filepath, "r") as f:
        for line in f:
            entry = json.loads(line)
            qid = entry["query_id"]
            # If it's a query record
            if "text" in entry:
                data[qid] = entry["text"].strip()
            else:
                # It's a qrel record
                if qid not in data:
                    data[qid] = {}
                doc_id = entry["doc_id"]
                relevance = int(entry.get("relevance", 0))
                data[qid][doc_id] = relevance
    return data

queries = load_jsonl("trec_ct_2021_query.jsonl") 
qrels = load_jsonl("trec_ct_2021_qrels.jsonl")    


# Semantic-Only Search on tokenised embeddings
def semantic_search(query_text, top_k=10):
    """
    Search the entire index using a script_score
    with 'detailed_descrption_vector' and query vector.
    """
    query_vector = get_embedding(query_text)
    vector_query_body = {
        "size": top_k,  # put size in the body to avoid deprecation warnings
        "query": {
            "script_score": {
                "query": {
                    "bool": {
                        "must": [
                            {"exists": {"field": "detailed_descrption_vector"}}
                        ]
                    }
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'detailed_descrption_vector') + 1.0",
                    "params": {"query_vector": query_vector}
                }
            }
        }
    }

    response = es.search(index="ir_dev_index", body=vector_query_body)
    # Return doc_id from _source if that's how Qrels references them
    print("Time taken for semantic search: {} ms".format(response["took"]))
    return [hit["_source"]["doc_id"] for hit in response["hits"]["hits"]]

K = 10               # top K for retrieval & evaluation
RELEVANCE_THRESHOLD = 2  # Consider docs with relevance >= x as relevant when evaluating

def get_relevant_docs(qrel_dict, relevance_threshold=1):
    return {doc_id: rel for doc_id, rel in qrel_dict.items() if rel >= relevance_threshold}

def get_all_docs(qrel_dict):
    """Return the entire dictionary of {doc_id: relevance}, including relevance=0."""
    return {doc_id: rel for doc_id, rel in qrel_dict.items()}

all_metrics = []

for qid, query_text in tqdm(queries.items(), total=len(queries)):
    if not query_text:
        continue
    
    # Track the total time
    start_time = time.time()

    # Retrieve top K docs via semantic search
    retrieved_docs = semantic_search(query_text, top_k=K)
    
    # Filter qrels to get only docs with >= threshold ( for evaluation only )
    relevant_docs = get_relevant_docs(qrels.get(qid, {}), relevance_threshold=RELEVANCE_THRESHOLD)
    
    #All docs retrieved (i.e no threshold) to compute nDCG based on graded relevance 0,1,2
    ndcg_relevant_docs = get_all_docs(qrels.get(qid, {}))

    # Mark the end time for entire query processing
    total_time = time.time() - start_time

    # Compute metrics
    prec_val = precision_at_k(retrieved_docs, relevant_docs, K)
    rec_val = recall_at_k(retrieved_docs, relevant_docs, K)
    f_val = f1_at_k(retrieved_docs, relevant_docs, K)
    ap_val = average_precision(retrieved_docs, relevant_docs)
    ndcg_val = ndcg_at_k(retrieved_docs, ndcg_relevant_docs, K)
    mrr_val = mrr_at_k(retrieved_docs, relevant_docs, K)

    metrics = {
        "query_id": qid,
        "precision": prec_val,
        "recall": rec_val,
        "f1": f_val,
        "average_precision": ap_val,
        "ndcg": ndcg_val,
        "mrr": mrr_val,
        "time_seconds": total_time
    }
    all_metrics.append(metrics)
    print(f"Query {qid} retrieved: {retrieved_docs}")
    print(f" Query {qid} metrics: Precision@10: {prec_val:.4f}, Recall@10: {rec_val:.4f}, F1@10: {f_val:.4f}, AP: {ap_val:.4f}, nDCG@10: {ndcg_val:.4f}")
    print(f"  Total time for query: {total_time:.2f} seconds")

# Compute overall metrics
if all_metrics:
    mean_precision = np.mean([m["precision"] for m in all_metrics])
    mean_recall = np.mean([m["recall"] for m in all_metrics])
    mean_f1 = np.mean([m["f1"] for m in all_metrics])
    mean_ap = np.mean([m["average_precision"] for m in all_metrics])
    mean_ndcg = np.mean([m["ndcg"] for m in all_metrics])
    mean_mrr = np.mean([m["mrr"] for m in all_metrics])
    avg_time = np.mean([m["time_seconds"] for m in all_metrics])

    print("\n--- Overall Evaluation (Token Clinical Bert Semantic-Only) ---")
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Date/time:{current_time}")
    print(f"Relevance threshold >= {RELEVANCE_THRESHOLD}")
    print(f"Mean Precision@{K}: {mean_precision:.4f}")
    print(f"Mean Recall@{K}: {mean_recall:.4f}")
    print(f"Mean F1@{K}: {mean_f1:.4f}")
    print(f"Mean Average Precision (MAP): {mean_ap:.4f}")
    print(f"Mean nDCG@{K}: {mean_ndcg:.4f}")
    print(f"Mean MRR@{K}: {mean_mrr:.4f}")
    print(f"Average total time per query: {avg_time:.2f} seconds")
else:
    print("No queries processed.")

# Create a DataFrame to export the computed results for all queries
data = {
    "Test": "Token Emdedding Only.",
    "Date/Time": [current_time],
    "Relevance Threshold": [RELEVANCE_THRESHOLD],
    f"Mean Precision@{K}": [mean_precision],
    f"Mean Recall@{K}": [mean_recall],
    f"Mean F1@{K}": [mean_f1],
    "Mean Average Precision (MAP)": [mean_ap],
    f"Mean nDCG@{K}": [mean_ndcg],
    f"Mean MRR@{K}": [mean_mrr],
    "Avg Time per Query (s)": [avg_time],
}
df = pd.DataFrame(data)

# Export to an Excel file
xlsx_filename = "evaluation_results_TOKEN_embeddings_only.xlsx"
df.to_excel(xlsx_filename, index=False)

print(f"\nMetrics have been exported to {xlsx_filename}")

  4%|▍         | 3/75 [00:00<00:07, 10.20it/s]

Time taken for semantic search: 13 ms
Query 1 retrieved: ['NCT03201367', 'NCT04201002', 'NCT02910596', 'NCT04694053', 'NCT04450316', 'NCT04645264', 'NCT02776202', 'NCT04159077', 'NCT04829890', 'NCT04553562']
 Query 1 metrics: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000
  Total time for query: 0.11 seconds
Time taken for semantic search: 13 ms
Query 2 retrieved: ['NCT04061213', 'NCT00976625', 'NCT00478296', 'NCT02910349', 'NCT02316587', 'NCT04299971', 'NCT03863132', 'NCT04078867', 'NCT04830787', 'NCT00820352']
 Query 2 metrics: Precision@10: 0.1000, Recall@10: 0.0072, F1@10: 0.0135, AP: 0.2000, nDCG@10: 0.2146
  Total time for query: 0.10 seconds
Time taken for semantic search: 12 ms
Query 3 retrieved: ['NCT03902262', 'NCT02809651', 'NCT03377062', 'NCT02325154', 'NCT03080779', 'NCT01654198', 'NCT00395330', 'NCT03496545', 'NCT02097212', 'NCT00313703']
 Query 3 metrics: Precision@10: 0.1000, Recall@10: 0.0312, F1@10: 0.0476, AP: 0.3333, nDCG@10: 0.1

  7%|▋         | 5/75 [00:00<00:06, 10.34it/s]

Time taken for semantic search: 11 ms
Query 4 retrieved: ['NCT02407080', 'NCT03647631', 'NCT02291796', 'NCT00706641', 'NCT01281033', 'NCT03902262', 'NCT03201367', 'NCT00498238', 'NCT03712345', 'NCT02768259']
 Query 4 metrics: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000
  Total time for query: 0.10 seconds
Time taken for semantic search: 13 ms
Query 5 retrieved: ['NCT01050361', 'NCT03902262', 'NCT01905488', 'NCT01252875', 'NCT01281033', 'NCT04005222', 'NCT04299971', 'NCT02781207', 'NCT02961127', 'NCT01458860']
 Query 5 metrics: Precision@10: 0.1000, Recall@10: 0.0084, F1@10: 0.0155, AP: 0.1000, nDCG@10: 0.0857
  Total time for query: 0.09 seconds
Time taken for semantic search: 12 ms
Query 6 retrieved: ['NCT02200328', 'NCT02776202', 'NCT03902262', 'NCT00006132', 'NCT04614142', 'NCT00789997', 'NCT01678365', 'NCT02086825', 'NCT03378479', 'NCT00003805']
 Query 6 metrics: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0

 12%|█▏        | 9/75 [00:00<00:06, 10.42it/s]

...

Time taken for semantic search: 12 ms
Query 67 retrieved: ['NCT03647631', 'NCT02113514', 'NCT02797600', 'NCT01402583', 'NCT01116167', 'NCT03630926', 'NCT00055991', 'NCT03144037', 'NCT04553562', 'NCT01095198']
 Query 67 metrics: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0212
  Total time for query: 0.09 seconds
Time taken for semantic search: 12 ms
Query 68 retrieved: ['NCT00751426', 'NCT01408836', 'NCT01561248', 'NCT04622449', 'NCT00922883', 'NCT01360268', 'NCT02776202', 'NCT01410578', 'NCT00380614', 'NCT02464124']
 Query 68 metrics: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0367
  Total time for query: 0.09 seconds
Time taken for semantic search: 13 ms
Query 69 retrieved: ['NCT00021866', 'NCT03144037', 'NCT03867240', 'NCT01276041', 'NCT01754493', 'NCT03640273', 'NCT00971074', 'NCT01099657', 'NCT00009672', 'NCT03498742']
 Query 69 metrics: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@1

 95%|█████████▍| 71/75 [00:06<00:00, 10.46it/s]

Time taken for semantic search: 13 ms
Query 70 retrieved: ['NCT00908076', 'NCT00185575', 'NCT00053846', 'NCT00608764', 'NCT04051060', 'NCT04573504', 'NCT00395330', 'NCT00848406', 'NCT00313703', 'NCT00449865']
 Query 70 metrics: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000
  Total time for query: 0.10 seconds
Time taken for semantic search: 11 ms
Query 71 retrieved: ['NCT00105066', 'NCT04357119', 'NCT02556749', 'NCT00640757', 'NCT00003906', 'NCT01215136', 'NCT01707771', 'NCT01977417', 'NCT01281033', 'NCT00456885']
 Query 71 metrics: Precision@10: 0.2000, Recall@10: 0.0150, F1@10: 0.0280, AP: 0.3929, nDCG@10: 0.2334
  Total time for query: 0.09 seconds
Time taken for semantic search: 4 ms
Query 72 retrieved: ['NCT03902262', 'NCT00105066', 'NCT03201367', 'NCT04352062', 'NCT00380614', 'NCT02086994', 'NCT03117010', 'NCT00613093', 'NCT04413149', 'NCT03212872']
 Query 72 metrics: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10

100%|██████████| 75/75 [00:07<00:00, 10.30it/s]

Time taken for semantic search: 4 ms
Query 73 retrieved: ['NCT03902262', 'NCT01795131', 'NCT02325492', 'NCT00676247', 'NCT04121572', 'NCT04850547', 'NCT04497012', 'NCT01385098', 'NCT04531163', 'NCT00692224']
 Query 73 metrics: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0528
  Total time for query: 0.09 seconds
Time taken for semantic search: 12 ms
Query 74 retrieved: ['NCT03479190', 'NCT02501395', 'NCT03500861', 'NCT04130204', 'NCT01875458', 'NCT03453632', 'NCT02541292', 'NCT00111241', 'NCT00449865', 'NCT00971074']
 Query 74 metrics: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000
  Total time for query: 0.10 seconds
Time taken for semantic search: 13 ms
Query 75 retrieved: ['NCT00908076', 'NCT04051060', 'NCT00493974', 'NCT00449865', 'NCT00936585', 'NCT02896816', 'NCT00646204', 'NCT03446807', 'NCT01366027', 'NCT04130204']
 Query 75 metrics: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10

### BM25 and semantic search with ClinicalBERT

Initial retrieval with BM25 and re-ranking with ClinicalBERT

In [ ]:
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import time
import pandas as pd
from datetime import datetime

# Load ClinicalBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModel.from_pretrained("medicalai/ClinicalBERT")


def mean_pooling(model_output, attention_mask):
    """
    Perform mean pooling on token embeddings while taking the attention mask into account.
    """
    token_embeddings = model_output.last_hidden_state  # (batch_size, sequence_length, hidden_size)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
    return sum_embeddings / sum_mask

def get_embedding(text, max_length=512):
    """
    Compute and return the embedding for the given text using ClinicalBERT.
    Returns the embedding as a list of floats.
    """
    encoded_input = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    with torch.no_grad():
        model_output = model(**encoded_input)
    embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    return embedding[0].cpu().numpy().tolist()

# Evaluation Metric Functions
def precision_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / k if k else 0.0

def recall_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    total_relevant = sum(1 for doc, rel in relevant_docs.items() if rel > 0)
    if total_relevant == 0:
        return 0.0
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / total_relevant

def f1_at_k(re_ranked_docs, relevant_docs, k):
    p = precision_at_k(re_ranked_docs, relevant_docs, k)
    r = recall_at_k(re_ranked_docs, relevant_docs, k)
    if (p + r) == 0:
        return 0.0
    return 2 * (p * r) / (p + r)

def average_precision(re_ranked_docs, relevant_docs):
    if not relevant_docs:
        return 0.0
    
    relevant_indices = [
        i for i, doc in enumerate(re_ranked_docs)
        if doc in relevant_docs and relevant_docs[doc] > 0
    ]
    if not relevant_indices:
        return 0.0
    
    ap_sum = 0.0
    for idx, rank_pos in enumerate(relevant_indices, start=1):
        prec = precision_at_k(re_ranked_docs, relevant_docs, rank_pos+1)
        ap_sum += prec
    return ap_sum / len(relevant_indices)

def ndcg_at_k(re_ranked_docs, relevant_docs, k):
    def dcg(scores):
        return sum(( (2**s - 1) / np.log2(idx + 2)) for idx, s in enumerate(scores))
    
    top_k_docs = re_ranked_docs[:k]
    actual_scores = [relevant_docs[doc] if doc in relevant_docs else 0 for doc in top_k_docs]
    actual_dcg = dcg(actual_scores)
    
    sorted_rels = sorted(relevant_docs.values(), reverse=True)[:k]
    ideal_dcg = dcg(sorted_rels)
    
    if ideal_dcg == 0:
        return 0.0
    return actual_dcg / ideal_dcg

def mrr_at_k(re_ranked_docs, relevant_docs, k):
    for i, doc_id in enumerate(re_ranked_docs[:k], start=1):
        if doc_id in relevant_docs and relevant_docs[doc_id] > 0:
            return 1.0 / i
    return 0.0

# Load queries and qrels from JSONL files
def load_jsonl(filepath):
    data = {}
    with open(filepath, "r") as f:
        for line in f:
            entry = json.loads(line)
            key = entry["query_id"]
            if "text" in entry:
                data[key] = entry["text"].strip()
            else:
                if key not in data:
                    data[key] = {}
                data[key][entry["doc_id"]] = int(entry.get("relevance", 0))
    return data

# Stage 1: initial candidate retrieval based on text fields "title", "summary", "detailed_descrption
def initial_candidate_retrieval(query_text, candidate_size=500):
    initial_body = {
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["title", "summary", "detailed_descrption"]
            }
        }
    }
    response = es.search(index="ir_dev_index", body=initial_body, size=candidate_size)
    candidate_ids = [hit["_id"] for hit in response["hits"]["hits"]]
    print("Time taken to for lexical search: {} ms".format(response["took"]))
    return candidate_ids

# Stage 2: vector re-ranking on the candidate set.
def vector_re_ranking(query_text, candidate_ids, top_k=10):
    query_vector = get_embedding(query_text)  # Assumes get_embedding() is defined elsewhere.
    vector_query_body = {
        "query": {
            "script_score": {
                "query": {
                    "bool": {
                        "filter": [
                            {"ids": {"values": candidate_ids}},
                            {"exists": {"field": "detailed_descrption_vector"}}
                        ]
                    }
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'detailed_descrption_vector') + 1.0",
                    "params": {"query_vector": query_vector}
                }
            }
        }
    }
    response = es.search(index="ir_dev_index", body=vector_query_body, size=top_k)
    print("Time taken to re-rank: {} ms".format(response["took"]))
    return [hit["_source"]["doc_id"] for hit in response["hits"]["hits"]]

# Load queries and qrels
queries = load_jsonl("trec_ct_2021_query.jsonl")
qrels = load_jsonl("trec_ct_2021_qrels.jsonl") 

# Set the RELEVANCE_THRESHOLD. Consider any document with relevance >= RELEVANCE_THRESHOLD as relevant for evaluation purposes
RELEVANCE_THRESHOLD = 2
def get_relevant_docs(qrel_dict, relevance_threshold=1):
    return {doc_id: rel for doc_id, rel in qrel_dict.items() if rel >= RELEVANCE_THRESHOLD}

# For ndcg calculation
def get_all_docs(qrel_dict):
    """Return the entire dictionary of {doc_id: relevance}, including relevance=0."""
    return {doc_id: rel for doc_id, rel in qrel_dict.items()}

CANDIDATE_SIZE = 1000 #top records for first stage ranking
K = 10  # top K for re-ranking and evaluation
all_metrics = []

print("Processing queries and computing evaluation metrics...")
for qid, query_text in tqdm(queries.items(), total=len(queries)):
    if not query_text:
        continue

    # Track the total time
    start_time = time.time()
    
    # Stage 1: retrieve candidate document IDs.
    candidate_ids = initial_candidate_retrieval(query_text, candidate_size=CANDIDATE_SIZE)
    # Stage 2: re-rank candidates via vector search.
    retrieved_docs = vector_re_ranking(query_text, candidate_ids, top_k=K)
    
    # Mark the end time for entire query processing
    total_time = time.time() - start_time

    # Get relevant docs for this query from qrels using the RELEVANCE_THRESHOLD (for evaluation)
    relevant_docs = get_relevant_docs(qrels.get(qid, {}), relevance_threshold=RELEVANCE_THRESHOLD)
    
    #All docs retrieved (i.e no threshold) to compute nDCG based on graded relevance 0,1,2
    ndcg_relevant_docs = get_all_docs(qrels.get(qid, {}))
    
    # Compute evaluation measures using our functions.
    prec = precision_at_k(retrieved_docs, relevant_docs, K)
    rec = recall_at_k(retrieved_docs, relevant_docs, K)
    f1 = f1_at_k(retrieved_docs, relevant_docs, K)
    ap = average_precision(retrieved_docs, relevant_docs)
    ndcg = ndcg_at_k(retrieved_docs, ndcg_relevant_docs, K)
    mrr_val = mrr_at_k(retrieved_docs, relevant_docs, K)
    
    metrics = {
        "query_id": qid,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "average_precision": ap,
        "ndcg": ndcg,
        "mrr": mrr_val,
        "time_seconds": total_time
    }
    all_metrics.append(metrics)
    
    print(f"Query {qid}: Precision@{K}: {prec:.4f}, Recall@{K}: {rec:.4f}, F1@{K}: {f1:.4f}, AP: {ap:.4f}, nDCG@{K}: {ndcg:.4f}, mrr@{K}: {mrr_val:.4f}")
    print(f"  Total time for query: {total_time:.2f} seconds")

# Compute overall evaluation metrics.
if all_metrics:
    avg_precision = np.mean([m["precision"] for m in all_metrics])
    avg_recall = np.mean([m["recall"] for m in all_metrics])
    avg_f1 = np.mean([m["f1"] for m in all_metrics])
    mean_ap = np.mean([m["average_precision"] for m in all_metrics])
    mean_ndcg = np.mean([m["ndcg"] for m in all_metrics])
    avg_mrr = np.mean([m["mrr"] for m in all_metrics])
    avg_time = np.mean([m["time_seconds"] for m in all_metrics])
    
    
    print("\nBM25L + Semantic search (Clinical BERT) - Overall evaluation metrics:")
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Date/time:{current_time}")
    print(f"\nThreshold =: {RELEVANCE_THRESHOLD}")
    print(f"Mean Precision@{K}: {avg_precision:.4f}")
    print(f"Mean Recall@{K}: {avg_recall:.4f}")
    print(f"Mean F1@{K}: {avg_f1:.4f}")
    print(f"Mean Average Precision (MAP): {mean_ap:.4f}")
    print(f"Mean nDCG@{K}: {mean_ndcg:.4f}")
    print(f"Mean MRR@{K}: {avg_mrr:.4f}")
    print(f"Average total time per query: {avg_time:.2f} seconds")
else:
    print("No queries processed.")

# Create a DataFrame to export the computed results for all queries
data = {
    "Test": "BM25L and Token Emdedding.",
    "Date/Time": [current_time],
    "Relevance Threshold": [RELEVANCE_THRESHOLD],
    f"Mean Precision@{K}": [avg_precision],
    f"Mean Recall@{K}": [avg_recall],
    f"Mean F1@{K}": [avg_f1],
    "Mean Average Precision (MAP)": [mean_ap],
    f"Mean nDCG@{K}": [mean_ndcg],
    f"Mean MRR@{K}": [avg_mrr],
    "Avg Time per Query (s)": [avg_time],
}
df = pd.DataFrame(data)

# Export to an Excel file
xlsx_filename = "evaluation_results_Lexical_and_TOKEN_embeddings.xlsx"
df.to_excel(xlsx_filename, index=False)

print(f"\nMetrics have been exported to {xlsx_filename}")


Processing queries and computing evaluation metrics...


  0%|          | 0/75 [00:00<?, ?it/s]/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_54465/781688488.py:128: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_index", body=initial_body, size=candidate_size)
/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_54465/781688488.py:154: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_index", body=vector_query_body, size=top_k)
  1%|▏         | 1/75 [00:01<02:09,  1.75s/it]

Time taken to for lexical search: 1314 ms
Time taken to re-rank: 60 ms
Query 1: Precision@10: 0.1000, Recall@10: 0.0213, F1@10: 0.0351, AP: 0.1667, nDCG@10: 0.0784, mrr@10: 0.1667
  Total time for query: 1.75 seconds
Time taken to for lexical search: 587 ms


  3%|▎         | 2/75 [00:03<01:47,  1.47s/it]

Time taken to re-rank: 325 ms
Query 2: Precision@10: 0.1000, Recall@10: 0.0072, F1@10: 0.0135, AP: 0.2000, nDCG@10: 0.2146, mrr@10: 0.2000
  Total time for query: 1.27 seconds


...

Time taken to for lexical search: 86 ms
Time taken to re-rank: 16 ms
Query 73: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0537, mrr@10: 0.0000
  Total time for query: 0.42 seconds


 99%|█████████▊| 74/75 [00:37<00:00,  2.31it/s]

Time taken to for lexical search: 103 ms
Time taken to re-rank: 17 ms
Query 74: Precision@10: 0.1000, Recall@10: 0.0909, F1@10: 0.0952, AP: 0.2000, nDCG@10: 0.0851, mrr@10: 0.2000
  Total time for query: 0.45 seconds


100%|██████████| 75/75 [00:38<00:00,  1.95it/s]

Time taken to for lexical search: 98 ms
Time taken to re-rank: 15 ms
Query 75: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0245, mrr@10: 0.0000
  Total time for query: 0.43 seconds

BM25L + Semantic search (Clinical BERT) - Overall evaluation metrics:
Date/time:2025-04-10 08:17:38

Threshold =: 2
Mean Precision@10: 0.0773
Mean Recall@10: 0.0122
Mean F1@10: 0.0191
Mean Average Precision (MAP): 0.1695
Mean nDCG@10: 0.1076
Mean MRR@10: 0.1858
Average total time per query: 0.51 seconds


### Lexical search only (elastic saerch BM25)

In [ ]:
import json
import numpy as np
from elasticsearch import Elasticsearch, helpers
from tqdm import tqdm
import time
import pandas as pd
from datetime import datetime

# Evaluation Metric Functions
def precision_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / k if k else 0.0

def recall_at_k(re_ranked_docs, relevant_docs, k):
    top_k_docs = re_ranked_docs[:k]
    total_relevant = sum(1 for doc, rel in relevant_docs.items() if rel > 0)
    if total_relevant == 0:
        return 0.0
    num_relevant_in_k = sum(1 for doc in top_k_docs 
                            if doc in relevant_docs and relevant_docs[doc] > 0)
    return num_relevant_in_k / total_relevant

def f1_at_k(re_ranked_docs, relevant_docs, k):
    p = precision_at_k(re_ranked_docs, relevant_docs, k)
    r = recall_at_k(re_ranked_docs, relevant_docs, k)
    if (p + r) == 0:
        return 0.0
    return 2 * (p * r) / (p + r)

def average_precision(re_ranked_docs, relevant_docs):
    if not relevant_docs:
        return 0.0
    
    relevant_indices = [
        i for i, doc in enumerate(re_ranked_docs)
        if doc in relevant_docs and relevant_docs[doc] > 0
    ]
    if not relevant_indices:
        return 0.0
    
    ap_sum = 0.0
    for idx, rank_pos in enumerate(relevant_indices, start=1):
        prec = precision_at_k(re_ranked_docs, relevant_docs, rank_pos+1)
        ap_sum += prec
    return ap_sum / len(relevant_indices)

def ndcg_at_k(re_ranked_docs, relevant_docs, k):
    def dcg(scores):
        return sum(( (2**s - 1) / np.log2(idx + 2)) for idx, s in enumerate(scores))
    
    top_k_docs = re_ranked_docs[:k]
    actual_scores = [relevant_docs[doc] if doc in relevant_docs else 0 for doc in top_k_docs]
    actual_dcg = dcg(actual_scores)
    
    sorted_rels = sorted(relevant_docs.values(), reverse=True)[:k]
    ideal_dcg = dcg(sorted_rels)
    
    if ideal_dcg == 0:
        return 0.0
    return actual_dcg / ideal_dcg

def mrr_at_k(re_ranked_docs, relevant_docs, k):
    for i, doc_id in enumerate(re_ranked_docs[:k], start=1):
        if doc_id in relevant_docs and relevant_docs[doc_id] > 0:
            return 1.0 / i
    return 0.0

# Define the text-based query search 
def search_text_based(query_text, K=10):
    search_body = {
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["title", "summary", "detailed_descrption"]
            }
        }
    }
    response = es.search(index="ir_dev_index", body=search_body, size=K)
    print("Time taken to retrieve (ES only): {} ms".format(response["took"]))
    return [hit["_source"]["doc_id"] for hit in response['hits']['hits']]

# Load queries from a JSONL file   
def load_queries(file_path):
    queries = {}
    with open(file_path, "r") as f:
        for line in f:
            doc = json.loads(line)
            queries[doc["query_id"]] = doc["text"].strip()
    return queries

# Load qrels from a JSONL file
def load_qrels(file_path):
    qrels = {}
    with open(file_path, "r") as f:
        for line in f:
            doc = json.loads(line)
            qid = doc["query_id"]
            relevance = int(doc.get("relevance", 0))
            # Only keep documents meeting the threshold; else, assign 0.
            relevance = relevance if relevance >= RELEVANCE_THRESHOLD else 0
            if qid not in qrels:
                qrels[qid] = {}
            qrels[qid][doc["doc_id"]] = relevance
    return qrels

RELEVANCE_THRESHOLD = 2  # Only documents with relevance >= x are considered relevant for evaluation
K = 10  # Top-K documents for evaluation

# Filter qrels based on relevance threshold (for evaluation)
def get_relevant_docs(qrel_dict, threshold=RELEVANCE_THRESHOLD):
    return {doc_id: rel for doc_id, rel in qrel_dict.items() if rel >= threshold}

# For ndcg calculation
def get_all_docs(qrel_dict):
    """Return the entire dictionary of {doc_id: relevance}, including relevance=0."""
    return {doc_id: rel for doc_id, rel in qrel_dict.items()}

if __name__ == "__main__":
    # Set file paths for your queries and qrels JSONL files.
    queries_file = "trec_ct_2021_query.jsonl"
    qrels_file = "trec_ct_2021_qrels.jsonl"

    # Load queries and qrels.
    queries = load_queries(queries_file)
    qrels = load_qrels(qrels_file)

    all_metrics = []

    print("BM25L - Processing queries and computing evaluation metrics...")
    for qid, qtext in tqdm(queries.items(), total=len(queries)):
        if not qtext:
            continue
        
        # Track the total time
        start_time = time.time()
        # Stage 1: Retrieve top-K documents using text-based search.
        retrieved_docs = search_text_based(qtext, K=K)

        # Mark the end time for entire query processing
        total_time = time.time() - start_time

        # Get relevant docs for this query using the relevance threshold.
        relevant_docs = get_relevant_docs(qrels.get(qid, {}), threshold=RELEVANCE_THRESHOLD)

        #All docs retrieved (i.e no threshold) to compute nDCG based on graded relevance 0,1,2
        ndcg_relevant_docs = get_all_docs(qrels.get(qid, {}))

        # Compute evaluation measures.
        prec = precision_at_k(retrieved_docs, relevant_docs, K)
        rec = recall_at_k(retrieved_docs, relevant_docs, K)
        f1 = f1_at_k(retrieved_docs, relevant_docs, K)
        ap = average_precision(retrieved_docs, relevant_docs)
        ndcg = ndcg_at_k(retrieved_docs, ndcg_relevant_docs, K)
        mrr_val = mrr_at_k(retrieved_docs, relevant_docs, K)
        
        metrics = {
            "query_id": qid,
            "precision": prec,
            "recall": rec,
            "f1": f1,
            "average_precision": ap,
            "ndcg": ndcg,
            "mrr": mrr_val,
            "time_seconds": total_time
        }
        all_metrics.append(metrics)
        print(f"Query {qid}: Precision@{K}: {prec:.4f}, Recall@{K}: {rec:.4f}, F1@{K}: {f1:.4f}, AP: {ap:.4f}, nDCG@{K}: {ndcg:.4f}, mrr{K}: {mrr_val:.4f}")
        print(f"  Total time for query: {total_time:.2f} seconds")

    # Compute overall evaluation metrics.
    if all_metrics:
        avg_precision = np.mean([m["precision"] for m in all_metrics])
        avg_recall = np.mean([m["recall"] for m in all_metrics])
        avg_f1 = np.mean([m["f1"] for m in all_metrics])
        mean_ap = np.mean([m["average_precision"] for m in all_metrics])
        mean_ndcg = np.mean([m["ndcg"] for m in all_metrics])
        mean_mrr = np.mean([m["mrr"] for m in all_metrics])
        avg_time = np.mean([m["time_seconds"] for m in all_metrics])

        print("\nBM25L(standard es) - Overall evaluation metrics:")
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Date/time:{current_time}")
        print(f"\nThreshold = {RELEVANCE_THRESHOLD}")
        print(f"Mean Precision@{K}: {avg_precision:.4f}")
        print(f"Mean Recall@{K}: {avg_recall:.4f}")
        print(f"Mean F1@{K}: {avg_f1:.4f}")
        print(f"Mean Average Precision (MAP): {mean_ap:.4f}")
        print(f"Mean nDCG@{K}: {mean_ndcg:.4f}")
        print(f"Mean MRR@{K}: {mean_mrr:.4f}")
        print(f"Average total time per query: {avg_time:.2f} seconds")
    else:
        print("No queries processed.")

# Create a DataFrame to export the computed results for all queries
data = {
    "Test": "BM25L Only.",
    "Date/Time": [current_time],
    "Relevance Threshold": [RELEVANCE_THRESHOLD],
    f"Mean Precision@{K}": [avg_precision],
    f"Mean Recall@{K}": [avg_recall],
    f"Mean F1@{K}": [avg_f1],
    "Mean Average Precision (MAP)": [mean_ap],
    f"Mean nDCG@{K}": [mean_ndcg],
    f"Mean MRR@{K}": [mean_mrr],
    "Avg Time per Query (s)": [avg_time],
}
df = pd.DataFrame(data)

# Export to an Excel file
xlsx_filename = "BM25L_only.xlsx"
df.to_excel(xlsx_filename, index=False)

print(f"\nMetrics have been exported to {xlsx_filename}")

BM25L - Processing queries and computing evaluation metrics...


  0%|          | 0/75 [00:00<?, ?it/s]/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_28729/2568875947.py:80: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_index", body=search_body, size=K)
  3%|▎         | 2/75 [00:00<00:04, 16.50it/s]

Time taken to retrieve (ES only): 46 ms
Query 1: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
  Total time for query: 0.06 seconds
Time taken to retrieve (ES only): 46 ms
Query 2: Precision@10: 0.2000, Recall@10: 0.0145, F1@10: 0.0270, AP: 0.1964, nDCG@10: 0.1428, mrr10: 0.1429
  Total time for query: 0.06 seconds
Time taken to retrieve (ES only): 37 ms
Query 3: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
  Total time for query: 0.05 seconds


  5%|▌         | 4/75 [00:00<00:04, 16.89it/s]

Time taken to retrieve (ES only): 49 ms
Query 4: Precision@10: 0.1000, Recall@10: 0.0227, F1@10: 0.0370, AP: 0.1667, nDCG@10: 0.0784, mrr10: 0.1667
  Total time for query: 0.06 seconds


...

Time taken to retrieve (ES only): 39 ms
Query 65: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
  Total time for query: 0.06 seconds
Time taken to retrieve (ES only): 29 ms
Query 66: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
  Total time for query: 0.04 seconds
Time taken to retrieve (ES only): 17 ms
Query 67: Precision@10: 0.1000, Recall@10: 0.0167, F1@10: 0.0286, AP: 0.5000, nDCG@10: 0.1389, mrr10: 0.5000
  Total time for query: 0.03 seconds
Time taken to retrieve (ES only): 26 ms
Query 68: Precision@10: 0.1000, Recall@10: 0.0233, F1@10: 0.0377, AP: 0.2500, nDCG@10: 0.0948, mrr10: 0.2500
  Total time for query: 0.04 seconds
Time taken to retrieve (ES only): 27 ms
Query 69: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
  Total time for query: 0.04 seconds


 96%|█████████▌| 72/75 [00:04<00:00, 19.54it/s]

Time taken to retrieve (ES only): 55 ms
Query 70: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
  Total time for query: 0.07 seconds
Time taken to retrieve (ES only): 35 ms
Query 71: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
  Total time for query: 0.05 seconds
Time taken to retrieve (ES only): 37 ms
Query 72: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
  Total time for query: 0.05 seconds
Time taken to retrieve (ES only): 40 ms
Query 73: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
  Total time for query: 0.06 seconds


100%|██████████| 75/75 [00:04<00:00, 17.58it/s]

Time taken to retrieve (ES only): 37 ms
Query 74: Precision@10: 0.2000, Recall@10: 0.1818, F1@10: 0.1905, AP: 0.7500, nDCG@10: 0.3149, mrr10: 1.0000
  Total time for query: 0.08 seconds
Time taken to retrieve (ES only): 32 ms
Query 75: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
  Total time for query: 0.05 seconds

BM25L(standard es) - Overall evaluation metrics:
Date/time:2025-03-25 12:00:09

Threshold = 2
Mean Precision@10: 0.1093
Mean Recall@10: 0.0208
Mean F1@10: 0.0312
Mean Average Precision (MAP): 0.2424
Mean nDCG@10: 0.1164
Mean MRR@10: 0.2740
Average total time per query: 0.06 seconds

Metrics have been exported to BM25L_only.xlsx


### BM25W Description boosted ^2

In Elasticsearch, putting more weight on the description field

In [ ]:
import json
import numpy as np
from elasticsearch import Elasticsearch, helpers
from tqdm import tqdm
import time
import pandas as pd
from datetime import datetime

# Evaluation Functions
def precision_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    total_relevant = len([doc for doc, rel in relevant.items() if rel > 0])
    if total_relevant == 0:
        return 0.0
    correct = sum(1 for doc in retrieved_k if doc in relevant and relevant[doc] > 0)
    return correct / k

def recall_at_k(retrieved, relevant, k):
    retrieved_k = retrieved[:k]
    total_relevant = len([doc for doc, rel in relevant.items() if rel > 0])
    if total_relevant == 0:
        return 0.0
    correct = sum(1 for doc in retrieved_k if doc in relevant and relevant[doc] > 0)
    return correct / total_relevant

def f1_at_k(retrieved, relevant, k):
    p = precision_at_k(retrieved, relevant, k)
    r = recall_at_k(retrieved, relevant, k)
    if p + r == 0:
        return 0.0
    return 2 * p * r / (p + r)

def mrr_at_k(retrieved, relevant, k):
    for i, doc_id in enumerate(retrieved_docs[:k], start=1):
        if doc_id in relevant_docs and relevant_docs[doc_id] > 0:
            return 1.0 / i
    return 0.0

def average_precision(retrieved, relevant):
    if not relevant_docs:
        return 0.0
    
    relevant_indices = [
        i for i, doc in enumerate(retrieved_docs)
        if doc in relevant_docs and relevant_docs[doc] > 0
    ]
    if not relevant_indices:
        return 0.0
    
    ap_sum = 0.0
    for idx, rank_pos in enumerate(relevant_indices, start=1):
        prec = precision_at_k(retrieved_docs, relevant_docs, rank_pos+1)
        ap_sum += prec
    return ap_sum / len(relevant_indices)

def ndcg_at_k(retrieved, relevant, k):
    def dcg(scores):
        return sum((2**rel - 1) / np.log2(i + 1) for i, rel in enumerate(scores, start=1))
    retrieved_scores = [relevant.get(doc, 0) for doc in retrieved[:k]]
    ideal_scores = sorted(relevant.values(), reverse=True)[:k]
    if not ideal_scores or dcg(ideal_scores) == 0:
        return 0.0
    return dcg(retrieved_scores) / dcg(ideal_scores)

# Define the text-based query search
def search_text_based(query_text, K=10):
    search_body = {
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["title", "summary", "detailed_descrption^2"]
            }
        }
    }
    response = es.search(index="ir_dev_index", body=search_body, size=K)
    # Return document IDs stored in _source["doc_id"]
    return [hit["_source"]["doc_id"] for hit in response['hits']['hits']]

# --- Load queries from a JSONL file ---
def load_queries(file_path):
    queries = {}
    with open(file_path, "r") as f:
        for line in f:
            doc = json.loads(line)
            queries[doc["query_id"]] = doc["text"].strip()
    return queries

# Load qrels from a JSONL file 
def load_qrels(file_path):
    qrels = {}
    with open(file_path, "r") as f:
        for line in f:
            doc = json.loads(line)
            qid = doc["query_id"]
            relevance = int(doc.get("relevance", 0))
            # Only keep documents meeting the threshold; else, assign 0. (for evaluation only)
            relevance = relevance if relevance >= RELEVANCE_THRESHOLD else 0
            if qid not in qrels:
                qrels[qid] = {}
            qrels[qid][doc["doc_id"]] = relevance
    return qrels

RELEVANCE_THRESHOLD = 2  # Only documents with relevance >= 2 are considered relevant during evaluation
K = 10  # Top-K documents for evaluation

# Filter qrels based on relevance threshold for evaluation
def get_relevant_docs(qrel_dict, threshold=RELEVANCE_THRESHOLD):
    return {doc_id: rel for doc_id, rel in qrel_dict.items() if rel >= threshold}

# For ndcg calculation
def get_all_docs(qrel_dict):
    """Return the entire dictionary of {doc_id: relevance}, including relevance=0."""
    return {doc_id: rel for doc_id, rel in qrel_dict.items()}


if __name__ == "__main__":
    queries_file = "trec_ct_2021_query.jsonl"
    qrels_file = "trec_ct_2021_qrels.jsonl"

    # Load queries and qrels.
    queries = load_queries(queries_file)
    qrels = load_qrels(qrels_file)

    all_metrics = []

    print("BM25LW - Processing queries and computing evaluation metrics...")
    for qid, qtext in tqdm(queries.items(), total=len(queries)):
        if not qtext:
            continue
        
         #Track total time
        start_time = time.time()
        
        # Retrieve top-K documents using text-based search.
        retrieved_docs = search_text_based(qtext, K=K)
        
        # Mark the end time for entire query processing
        total_time = time.time() - start_time

        # Get relevant docs for this query using the relevance threshold.
        relevant_docs = get_relevant_docs(qrels.get(qid, {}), threshold=RELEVANCE_THRESHOLD)

        #All docs retrieved (i.e no threshold) to compute nDCG based on graded relevance 0,1,2
        ndcg_relevant_docs = get_all_docs(qrels.get(qid, {}))
        
        # Compute evaluation measures.
        prec = precision_at_k(retrieved_docs, relevant_docs, K)
        rec = recall_at_k(retrieved_docs, relevant_docs, K)
        f1 = f1_at_k(retrieved_docs, relevant_docs, K)
        ap = average_precision(retrieved_docs, relevant_docs)
        ndcg = ndcg_at_k(retrieved_docs, ndcg_relevant_docs, K)
        mrr_val = mrr_at_k(retrieved_docs, relevant_docs, K)

        metrics = {
            "query_id": qid,
            "precision": prec,
            "recall": rec,
            "f1": f1,
            "average_precision": ap,
            "ndcg": ndcg,
            "mrr": mrr_val,
            "time_seconds": total_time
        }
        all_metrics.append(metrics)
        print(f"Query {qid}: Precision@{K}: {prec:.4f}, Recall@{K}: {rec:.4f}, F1@{K}: {f1:.4f}, AP: {ap:.4f}, nDCG@{K}: {ndcg:.4f}, mrr{K}: {mrr_val:.4f}")

    # Compute overall evaluation metrics.
    if all_metrics:
        avg_precision = np.mean([m["precision"] for m in all_metrics])
        avg_recall = np.mean([m["recall"] for m in all_metrics])
        avg_f1 = np.mean([m["f1"] for m in all_metrics])
        mean_ap = np.mean([m["average_precision"] for m in all_metrics])
        mean_ndcg = np.mean([m["ndcg"] for m in all_metrics])
        mean_mrr = np.mean([m["mrr"] for m in all_metrics])
        avg_time = np.mean([m["time_seconds"] for m in all_metrics])

        print("\nBM25LW - description boosted ^2 - Overall evaluation metrics:")
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Date/time:{current_time}")
        print(f"\nThreshold = {RELEVANCE_THRESHOLD}")
        print(f"Mean Precision@{K}: {avg_precision:.4f}")
        print(f"Mean Recall@{K}: {avg_recall:.4f}")
        print(f"Mean F1@{K}: {avg_f1:.4f}")
        print(f"Mean Average Precision (MAP): {mean_ap:.4f}")
        print(f"Mean nDCG@{K}: {mean_ndcg:.4f}")
        print(f"Mean MRR@{K}: {mean_mrr:.4f}")
        print(f"Average total time per query: {avg_time:.2f} seconds")
    else:
        print("No queries processed.")

    # Create a DataFrame to export the computed results for all queries
data = {
    "Test": "BM25LW Boosted",
    "Date/Time": [current_time],
    "Relevance Threshold": [RELEVANCE_THRESHOLD],
    f"Mean Precision@{K}": [avg_precision],
    f"Mean Recall@{K}": [avg_recall],
    f"Mean F1@{K}": [avg_f1],
    "Mean Average Precision (MAP)": [mean_ap],
    f"Mean nDCG@{K}": [mean_ndcg],
    f"Mean MRR@{K}": [mean_mrr],
    "Avg Time per Query (s)": [avg_time],
}
df = pd.DataFrame(data)

# Export to an Excel file
xlsx_filename = "BM25LW Boosted.xlsx"
df.to_excel(xlsx_filename, index=False)

print(f"\nMetrics have been exported to {xlsx_filename}")


BM25LW - Processing queries and computing evaluation metrics...


  0%|          | 0/75 [00:00<?, ?it/s]

Query 1: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000


/var/folders/b3/59dkzgnj0zxdhdx31d2ztm000000gn/T/ipykernel_28729/1950038217.py:82: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index="ir_dev_index", body=search_body, size=K)
  3%|▎         | 2/75 [00:00<00:04, 17.87it/s]

Query 2: Precision@10: 0.2000, Recall@10: 0.0145, F1@10: 0.0270, AP: 0.1964, nDCG@10: 0.1428, mrr10: 0.1429
Query 3: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000


...

Query 63: Precision@10: 0.1000, Recall@10: 0.0286, F1@10: 0.0444, AP: 0.2500, nDCG@10: 0.0948, mrr10: 0.2500
Query 64: Precision@10: 0.2000, Recall@10: 0.0308, F1@10: 0.0533, AP: 0.6429, nDCG@10: 0.2935, mrr10: 1.0000
Query 65: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
Query 66: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
Query 67: Precision@10: 0.2000, Recall@10: 0.0333, F1@10: 0.0571, AP: 0.3750, nDCG@10: 0.2083, mrr10: 0.5000


 92%|█████████▏| 69/75 [00:03<00:00, 20.29it/s]

Query 68: Precision@10: 0.1000, Recall@10: 0.0233, F1@10: 0.0377, AP: 0.2500, nDCG@10: 0.0948, mrr10: 0.2500
Query 69: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
Query 70: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
Query 71: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000


100%|██████████| 75/75 [00:04<00:00, 17.61it/s]

Query 72: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
Query 73: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000
Query 74: Precision@10: 0.2000, Recall@10: 0.1818, F1@10: 0.1905, AP: 0.7500, nDCG@10: 0.3149, mrr10: 1.0000
Query 75: Precision@10: 0.0000, Recall@10: 0.0000, F1@10: 0.0000, AP: 0.0000, nDCG@10: 0.0000, mrr10: 0.0000

BM25LW - description boosted ^2 - Overall evaluation metrics:
Date/time:2025-03-25 12:00:13

Threshold = 2
Mean Precision@10: 0.1093
Mean Recall@10: 0.0200
Mean F1@10: 0.0303
Mean Average Precision (MAP): 0.2562
Mean nDCG@10: 0.1186
Mean MRR@10: 0.2857
Average total time per query: 0.06 seconds

Metrics have been exported to BM25LW Boosted.xlsx
